In [1]:
from rap_db import *
from rap_clean import *
from rap_viz import line, verse_graph
from nltk.stem import *
import pandas as pd

/home/keenan/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
doom = art_load(['Doom'])['Doom']
chief = art_load(['Chief Keef'])['Chief Keef']

In [3]:
#verse count for each
print(len(doom.uniq_art_verses))
print(len(chief.uniq_art_verses))

112
148


SINGLE LINE METRICS (check for correlations with corr plot)
2. Unique words ratio (#uniq words/#of words) done
3. Avg word length sum(#of letters in all words)/(#all words) done
4. Avg unique word length sum(#of letters in all unqiue words)/(#all unique words) done
5. Polarity
6. Sentiment
7. Unique broad vowel sound ratio (#of unique broad sylbls/# of broad sylbls)
8. Unique near vowel sound ratio (#of unique near sylbls/# of near sylbls)
9. Unique exact vowel sound ratio (#of unique exact sylbls/# of exact sylbls)
10. Avg sylbls per word sum(#of sylbs in all words)/(#all words)
11. Avg unqiue sylbls per word sum(#of unique sylbs in all words)/(#all unique words)
12. Unique POS Ratio (#of unique POS)/(#all words)

In [4]:
class line_data():
    #mainly just assign attributes used in other methods
    def __init__(self, line_obj):
        #words
        self.all_word_objs = line_obj.word_objs
        self.all_word_strs = line_obj.words_as_strings
        self.all_uniq_word_strs = line_obj.uniq_words_as_strings
        #vowels
        self.ex_vwls = line_obj.vowel_sounds
        self.nr_vwls = [v[:2] for v in self.ex_vwls]
        self.brd_vwls = [v[:1] for v in self.ex_vwls]
        #this is going to add sometime but need to help with perfectly correlated features
        check = set()
        self.ex_vwls_uniqs = []
        for w in self.all_word_objs:
            if w.text.lower() not in check:
                check = check|{w.text.lower()}
                self.ex_vwls_uniqs.extend(list(zip(*w.matches))[1])
        self.nr_vwls_uniqs = [v[:2] for v in self.ex_vwls_uniqs]
        self.brd_vwls_uniqs = [v[:1] for v in self.ex_vwls_uniqs]
        
        self.gen_line_stem()
        self.gen_line_metrics()
                
        #want to run in multinomial and bernouli ways (one with frequency one with there not there binary)
    def gen_line_stem(self):
        stemmer = SnowballStemmer("english")
        words_stm = [stemmer.stem(w.lower()) for w in self.all_word_strs if stemmer.stem(w.lower()) not in stopwords.words('english')]
        self.all_stemmed_words = list(filter(None, words_stm))
        self.unique_stemmed_words = set(self.all_stemmed_words)
        
    #may want to add stemming if accuracy shitty
    def gen_line_metrics(self):
        #word based metrics
        self.metrics={'avg_wrd_len':sum(map(len,self.all_word_strs))/len(self.all_word_strs),
        'avg_unq_wrd_len':sum(map(len,self.all_uniq_word_strs))/len(self.all_uniq_word_strs),
        'unq_wrds_rat':len(self.all_uniq_word_strs)/len(self.all_word_strs),
                      
        #vowel based metrics
            #average vowel sounds per word
        'avg_wrd_vwls':len(self.ex_vwls)/len(self.all_word_strs),
            #average unique vowel sounds per word
        'avg_wrd_brd_unq_vwls':len(set(self.brd_vwls))/len(self.all_word_strs),
        'avg_wrd_nr_unq_vwls':len(set(self.nr_vwls))/len(self.all_word_strs),
        'avg_wrd_ex_unq_vwls':len(set(self.ex_vwls))/len(self.all_word_strs),
            #average unique vowel sounds per unique word
        'avg_unq_wrd_brd_unq_vwls':len(set(self.brd_vwls_uniqs))/len(self.all_uniq_word_strs),
        'avg_unq_wrd_nr_unq_vwls':len(set(self.nr_vwls_uniqs))/len(self.all_uniq_word_strs),
        'avg_unq_wrd_ex_unq_vwls':len(set(self.ex_vwls_uniqs))/len(self.all_uniq_word_strs),
                      
        #specialized metrics
        'pol':float(),
        'sen':float(),
        'uniq_pos_rat':float()}

In [5]:
def art_to_verse_graph(art_obj, inp_pop=False, inp_exc_line=True, inp_opto_type='near'):#opto stuff here
    ret_verse_graphs = []
    for s in art_obj.songs:
        for v in s.uniq_art_verses:
            verse_g = verse_graph(v, art_obj.name, s.name)
            verse_g.opto_matches(pop=inp_pop, exc_line=inp_exc_line, opto_type=inp_opto_type, record=False)
            ret_verse_graphs.append(verse_g)
    return ret_verse_graphs

In [6]:
def verse_graph_to_lines(verse_graph_obj):
    ret_lines = []
    for v_line in verse_graph_obj.ver_as_lines:
        if v_line.word_objs:
            line_data_obj = line_data(v_line)
            ret_lines.append(line_data_obj)
    return ret_lines

In [7]:
#line_count
all_doom_lines = [ver for verse_g in art_to_verse_graph(doom, inp_pop=2, inp_exc_line=False, inp_opto_type='exact') for ver in verse_graph_to_lines(verse_g)]
len(all_doom_lines)

2873

In [9]:
all_chief_lines = [ver for verse_g in art_to_verse_graph(chief, inp_pop=False, inp_exc_line=True, inp_opto_type='near') for ver in verse_graph_to_lines(verse_g)]
len(all_chief_lines)

2012

What you did
built stemmer

What you need to do next


Notes
each artist should have a list of rap_line objects. Each line should have a word object dictionary, the preserved words as list of strings and then stemmed words/with unique stuff

Long term
Train models using two different training methadologies
1. text bag of words (simply look at words in textand classify using a naive bayes, random forest, SVM)
2. make a row for every line based on the whiteboarded lingustic measures (def use svm, maybe random forest, maybe KNN)